### Libraries

In [16]:
import numpy as np
import math as ma
import matplotlib.pyplot as plt
import scipy.integrate

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter
from matplotlib import animation

from astropy import constants as con
from astropy import units as u

%matplotlib osx

### Classes

In [19]:
class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
    def leng(self):
        return np.sqrt(self.x**2 + self.y**2 + self.z**2)
    def __add__(self, vect):
        return Vector(self.x + vect.x, self.y + vect.y, self.z + vect.z)
    def __sub__(self, vect):
        return Vector(self.x - vect.x, self.y - vect.y, self.z - vect.z)
    def __mul__(self, scal):
        return Vector(self.x * scal, self.y * scal, self.z * scal)
    def __rmul__(self, scal):
        return self * scal
    def __truediv__(self, scal):
        return Vector(self.x / scal, self.y / scal, self.z / scal)
    def norm(self):
        if (self.x == 0 and self.y == 0) and self.z == 0:
            return Vector(0, 0, 0)
        else:
            return self / self.leng()
    def dot(self, vect):
        return self.x * vect.x + self.y * vect.y + self.z * vect.z
    def angle(self, vect):
        return np.arccos(self.dot(vect) / (self.leng() * vect.leng()))
    def cross(self, vect):
        i = self.y * vect.z - self.z * vect.y
        j = self.z * vect.x - self.x * vect.z 
        k = self.x * vect.y - self.y * vect.x
        return Vector(i, j, k)
    def rotate(self, axis, ang):
        if axis == 'x':
            i = self.x
            j = self.y * np.cos(ang) - self.z * np.sin(ang)
            k = self.y * np.sin(ang) + self.z * np.cos(ang)
        if axis == 'y':
            i = self.x * np.cos(ang) + self.z * np.sin(ang)
            j = self.y
            k = -self.x * np.sin(ang) + self.z * np.cos(ang)
        if axis == 'z':
            i = self.x * np.cos(ang) - self.y * np.sin(ang)
            j = self.x * np.sin(ang) + self.y * np.cos(ang)
            k = self.z
        return Vector(i, j, k)
    
class Body:
    def __init__(self, pos, vel, mass, ecent, color, size, linewidth, hori, vert, name, text_size):
            self.pos = pos #Units: AU
            self.vel = vel #Units: AU yr^-1
            self.m = mass #Units: solMass
            self.e = ecent
            self.c, self.s, self.lw = color, size, linewidth
            self.ha, self.va, self.n, self.t = hori, vert, name, text_size

### Defining Masses and Positions with System in XY-Plane

In [20]:
#Gravitational constant; Units: AU^3 SolMass^-1 yr^-2
G = con.G.to(u.AU**3 / (u.solMass * u.yr**2)).value

#Masses; Units: SolMass
mS1 = 0.399648343801 #Star 1: GJ 15 A
mS2 = 0.164568 #Star 2: LHS 4
mP1 = 36*u.earthMass.to(u.solMass) #Planet 1: GJ 15 Ac
mP2 = 3.03*u.earthMass.to(u.solMass) #Planet 2: GJ 15 Ab

#Planet eccentricity and semi-major axis
e1 = 0.27
e2 = 0.0940

a1 = 5.4
a2 = 0.072

#Position vectors wrt Star 1; Units: AU
rS1S1 = Vector(0, 0, 0)
rS1S2 = Vector(146, 0, 0)
rS1P1 = Vector(-a1 * (1 + e1), 0, 0)
rS1P2 = Vector(-a2 * (1 + e2), 0, 0)

#COM position wrt Star 1, Units: AU
com_x = (rS1S2.x * mS2 + rS1P1.x * mP1 + rS1P2.x * mP2) / (mS1 + mS2 + mP1 + mP2)
com_y = (rS1S2.y * mS2 + rS1P1.y * mP1 + rS1P2.y * mP2) / (mS1 + mS2 + mP1 + mP2)
com_z = (rS1S2.z * mS2 + rS1P1.z * mP1 + rS1P2.z * mP2) / (mS1 + mS2 + mP1 + mP2)

com_S1 = Vector(com_x, com_y, com_z)

#Position vectors wrt COM; Units: AU
rS1 = rS1S1 - com_S1
rS2 = rS1S2 - com_S1
rP1 = rS1P1 - com_S1
rP2 = rS1P2 - com_S1
com = com_S1 - com_S1

### Reorienting Plane of System and Defining Velocities
Note: Uses Kepler's third law for 2-bodies to calculate period, which assumes the system consists of a set of isolated 2-body problems (i.e. the second star's gravitational influence does not play a role in the period of the planets).

Note: If you change the obs vector in this cell, you have to rerun the above cell before running this one.

In [21]:
#Direction to "face-on observer"; Defines normal vector to the plane
obs = Vector(0, 1.834129745, 1).norm()

#Calculating phi and theta angles of obs vector in spherical coordinates
#phi represents orbital inclination of system when facing XY-plane (0 deg = face-on, 90 deg = edge-on)
if (obs.x == 0 and obs.y == 0):
    theta, phi = 0, 0
else:
    theta = np.arctan(obs.y / obs.x)
    phi = np.arctan(np.sqrt(obs.x**2 + obs.y**2) / obs.z)

print(phi, theta)

#Rotating plane of system to be perpendicular to obs vector
pl_norm = (Vector(0, 0, 1).rotate('y', phi)).rotate('z', theta)

rS1 = rS1.rotate('y', phi).rotate('z', theta)
rS2 = rS2.rotate('y', phi).rotate('z', theta)
rP1 = rP1.rotate('y', phi).rotate('z', theta)
rP2 = rP2.rotate('y', phi).rotate('z', theta)

#Period; Units: yr
pS = np.sqrt((4 * np.pi**2 * (rS1S2.leng())**3) / (G * (mS1 + mS2)))
pP1 = np.sqrt((4 * np.pi**2 * (a1)**3) / (G * (mS1 + mP1)))
pP2 = np.sqrt((4 * np.pi**2 * (a2)**3) / (G * (mS1 + mP2)))

#Orbital Velocity; Units: AU yr^-1
vS1 = (2 * np.pi * (rS1.leng() - com.leng()) / pS) * obs.cross(rS1).norm()
vS2 = (2 * np.pi * (rS2.leng() - com.leng()) / pS) * obs.cross(rS2).norm()
vP1 = ((2 * np.pi * a1) / pP1) * np.sqrt((1 - e1) / (1 + e1)) * obs.cross(rP1).norm()
vP2 = ((2 * np.pi * a2) / pP2) * np.sqrt((1 - e2) / (1 + e2)) * obs.cross(rP2).norm()

1.0716321606815322 1.5707963267948966


/var/folders/gd/s_6zj5zj6kj8tmgqhw01jcnh0000gn/T/ipykernel_90910/3500013544.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  theta = np.arctan(obs.y / obs.x)


### Bodies in the System

In [22]:
#Defining Bodies
s1 = Body(rS1, vS1, mS1, 0, 'maroon', 25, 1, 'left', 'bottom','GJ 15A', 8)
s2 = Body(rS2, vS2, mS2, 0, 'firebrick', 15, 1, 'left', 'bottom', 'LHS 4', 8)
p1 = Body(rP1, vP1, mP1, e1, 'slateblue', 5, 0.3, 'right', 'top', 'GJ 15 Ab', 5)
p2 = Body(rP2, vP2, mP2, e2, 'darkorange', 2, 0.3, 'right', 'top', 'GJ 15 Ac', 5)

stars = np.array([s1,s2])
planets = np.array([p1, p2])
bodies = np.concatenate([stars, planets])

### Gravitational Acceleration Function

In [23]:
def GravityEquation(init, t, masses):
    derivs = np.array([])
    
    body_indices = 6 * np.arange(0, len(init) / 6).astype(int)
    for i in body_indices:
        pos_i = Vector(init[i], init[i + 1], init[i + 2])
        vel_i = Vector(init[i + 3], init[i + 4], init[i + 5])
        derivs = np.append(derivs, [vel_i.x, vel_i.y, vel_i.z])
        
        acel = Vector(0, 0, 0)
        for j in body_indices:
            if (j != i):
                pos_j = Vector(init[j], init[j + 1], init[j + 2])
                dist_vect = pos_j - pos_i
                dist = dist_vect.leng()
                acel = acel + (G * masses[int(j / 6)] / (dist)**3) * dist_vect
        derivs = np.append(derivs, [acel.x, acel.y, acel.z])
    return derivs

### Solving for Positions

In [24]:
init_params = np.array([])
masses = np.array([])

for b in bodies:
    init_params = np.append(init_params, [b.pos.x, b.pos.y, b.pos.z, b.vel.x, b.vel.y, b.vel.z])
    masses = np.append(masses, b.m)

time_span = np.linspace(0, pS, 50000)

#Body Positions
sol = scipy.integrate.odeint(GravityEquation, init_params, time_span, args = (masses,))

#COM Positions
com_num = np.zeros((len(sol), 3))
for i in range(len(masses)):
    com_num += masses[i] * sol[:, (6 * i): (6 * i) + 3]
    
com_pos = com_num / np.sum(masses)

#Size of Plots
max_x = np.max(sol[:, ::6]) + 0.05
min_x = np.min(sol[:, ::6]) - 0.05
mid_x = (abs(max_x) + abs(min_x)) / 2
max_y = np.max(sol[:, 1::6]) + 0.05
min_y = np.min(sol[:, 1::6]) - 0.05
mid_y = (abs(max_y) + abs(min_y)) / 2
max_z = np.max(sol[:, 2::6]) + 0.05
min_z = np.min(sol[:, 2::6]) - 0.05
mid_z = (abs(max_z) + abs(min_z)) / 2

### 2D Plot of XY-Plane for Positions

In [25]:
fig = plt.figure(figsize = (10, 8))
ax = fig.add_subplot(111)
fig.set_facecolor('white')
ax.set_facecolor('white')

body_indices = 6 * np.arange(0, len(bodies)).astype(int)

for j in body_indices:
    ax.plot(sol[:, j], sol[:, j + 1], color = bodies[int(j / 6)].c, \
            linestyle = '-', linewidth = bodies[int(j / 6)].lw)
    ax.scatter(sol[-1, j], sol[-1, j + 1], color = bodies[int(j / 6)].c, \
            marker = 'o', s = bodies[int(j / 6)].s, label = bodies[int(j / 6)].n)
    ax.text(sol[-1, j], sol[-1, j + 1], bodies[int(j / 6)].n, \
            rotation = 20, color = 'black', ha = bodies[int(j / 6)].ha, va = bodies[int(j / 6)].va, \
            fontsize = bodies[int(j / 6)].t)
    
ax.plot(com_pos[:, 0], com_pos[:, 1], color = 'black', linestyle = '--')
ax.scatter(com_pos[0, 0], com_pos[0, 1], color = 'black', marker = 'x', s = 10)
ax.text(com_pos[0, 0], com_pos[0, 1], 'COM', \
        color = 'black', ha = 'right', va = 'bottom', fontsize = 5)

ax.set_title('GJ 15 Binary Star System', fontsize = 15)
ax.set_xlim(min_x - mid_x * 0.1, max_x + mid_x * 0.1)
ax.set_ylim(min_y - mid_y * 0.1, max_y + mid_y * 0.1)
ax.set_xlabel('X-axis (AU)', fontsize = 10)
ax.set_ylabel('Y-axis (AU)', fontsize = 10)

ax.set_xlim(-110, 110)
ax.set_ylim(-110, 110)

plt.tight_layout(w_pad = 5)
plt.legend()
plt.show()

### Radial Velocity Curve (Facing XY-Plane)

In [26]:
#Normal RV Curve
fig = plt.figure(figsize = (10, 8))
ax = fig.add_subplot(111)

ax.plot(time_span, np.zeros((len(time_span), 1)), color = 'black', linestyle = '--')

star_indices = 6 * np.arange(0, len(stars)).astype(int)
for j in star_indices:
    ax.plot(time_span, sol[:, j + 5], bodies[int(j / 6)].c, label = bodies[int(j / 6)].n)
    
ax.set_title('Radial Velocity vs Time', fontsize = 20)
ax.set_xlabel('Time (yrs)', fontsize = 15)
ax.set_ylabel('Radial Velocity (AU/yr)', fontsize = 15)
ax.set_xlim(0 - (pS * 0.05), pS + pS * 0.05)
ax.set_ylim(-0.3, 0.3)
ax.legend(fontsize = 10)

plt.show()

#Zoomed-In RV Curve
fig = plt.figure(figsize = (10, 8))
ax = fig.add_subplot(111)

ax.plot(time_span, np.zeros((len(time_span), 1)), color = 'black', linestyle = '--')
axins = ax.inset_axes([0.35, 0.10, 0.6, 0.6])

star_indices = 6 * np.arange(0, len(stars)).astype(int)

for j in star_indices:
    ax.plot(time_span, sol[:, j + 5], bodies[int(j / 6)].c, label = bodies[int(j / 6)].n) 
    axins.plot(time_span, sol[:, j + 5], bodies[int(j / 6)].c)
    
ax.set_title('Radial Velocity vs Time', fontsize = 20)
ax.set_xlabel('Time (yrs)', fontsize = 15)
ax.set_ylabel('Radial Velocity (AU/yr)', fontsize = 15)
ax.set_xlim(0 - (pS * 0.05), pS + (pS * 0.05))
ax.set_ylim(-0.3, 0.3)
ax.legend(fontsize = 10)

min_rvS1 = np.min(sol[:, 5])
min_tS1 = time_span[np.where(sol[:, 5] == min_rvS1)][0]
axins.set_xlim(min_tS1 - 100, min_tS1 + 100)
axins.set_ylim(min_rvS1 - 0.001, min_rvS1 + 0.01)
axins.tick_params(labelsize = 8)

ax.indicate_inset_zoom(axins, edgecolor = "black")

plt.show()

### 2D Animation

In [ ]:
metadata = dict(title = '2D animation', artist = 'Matplotlib')
writer = FFMpegWriter(fps = 50, metadata = metadata, bitrate = 200000)

fig = plt.figure(figsize = (10, 8))
ax = fig.add_subplot(111)
fig.set_facecolor('black')
ax.set_facecolor('black')

with writer.saving(fig, "2D_Animation_GJ15_System.mp4", dpi = 200):
    
    body_indices = 6 * np.arange(0, len(sol[0]) / 6).astype(int)
    
    for i in range(len(time_span)):
        
        ax.clear()
        
        for j in body_indices:
            ax.plot(sol[:i, j], sol[:i, j + 1], color = bodies[int(j / 6)].c, \
                    linestyle = '-', linewidth = bodies[int(j / 6)].lw)
            ax.scatter(sol[i, j], sol[i, j + 1], color = bodies[int(j / 6)].c, \
                       marker = 'o', s = bodies[int(j / 6)].s)
            ax.text(sol[i, j], sol[i, j + 1], bodies[int(j / 6)].n, \
                    rotation = 10, color = 'white', ha = bodies[int(j / 6)].ha, va = bodies[int(j / 6)].va, \
                    fontsize = bodies[int(j / 6)].t)
            
        ax.plot(com_pos[:i, 0], com_pos[:i, 1], color = 'white', linestyle = '--')
        ax.scatter(com_pos[i, 0], com_pos[i, 1], color = 'white', marker = 'x', s = 10)
        ax.text(com_pos[i, 0], com_pos[i, 1], 'COM', \
                rotation = 10, color = 'white', ha = 'left', va = 'bottom', fontsize = 5)
        
        plt.suptitle('GJ 15 Binary Star System', x = 0.513, y = 0.95, color = 'white', fontsize = 15)
        ax.set_title('Elasped Time: ' + str(float('%.6g' % time_span[i])) + ' yrs', color = 'white', fontsize  = 12)
        ax.set_xlim(min_x - mid_x * 0.1, max_x + mid_x * 0.1)
        ax.set_ylim(min_y - mid_y * 0.1, max_y + mid_y * 0.1)
        ax.set_xlabel('X-axis (AU)', color = 'white', fontsize = 10)
        ax.set_ylabel('Y-axis (AU)', color = 'white', fontsize = 10)
        ax.tick_params(colors = 'white', which = 'both')
        for spine in ax.spines.values():
            spine.set_edgecolor('white')
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()

### 3D Animation

In [29]:
metadata = dict(title = '3D animation', artist = 'Matplotlib')
writer = FFMpegWriter(fps = 50, metadata = metadata, bitrate = 200000)

fig = plt.figure(figsize = (10, 8))
ax = fig.add_subplot(111, projection = '3d')
fig.set_facecolor('black')
ax.set_facecolor('black') 

ax.view_init(30, 45)

with writer.saving(fig, "3D_Animation_GJ15_System.mp4", dpi = 200):
    
    body_indices = 6 * np.arange(0, len(bodies)).astype(int)
    
    for i in range(len(time_span)):
        
        ax.clear()
        
        for j in body_indices:
            ax.plot(sol[:i, j], sol[:i, j + 1], sol[:i, j + 2], color = bodies[int(j / 6)].c, \
                    linestyle = '-', linewidth = bodies[int(j / 6)].lw)
            ax.scatter(sol[i, j], sol[i, j + 1], sol[i, j + 2], color = bodies[int(j / 6)].c, \
                       marker = 'o', s = bodies[int(j / 6)].s)
            ax.text(sol[i, j], sol[i, j + 1], sol[i, j + 2], bodies[int(j / 6)].n, \
                    color = 'white', ha = bodies[int(j / 6)].ha, va = bodies[int(j / 6)].va, fontsize = bodies[int(j / 6)].t)
            
        ax.plot(com_pos[:i, 0], com_pos[:i, 1], com_pos[:i, 2], color = 'white', linestyle = '--')
        ax.scatter(com_pos[i, 0], com_pos[i, 1], com_pos[i, 2], color = 'white', marker = 'x', s = 10)
        ax.text(com_pos[i, 0], com_pos[i, 1], com_pos[i, 2], 'COM', \
                color = 'white', ha = 'left', va = 'bottom', fontsize = 5)
        
        plt.suptitle('GJ 15 Binary Star System', x = 0.513, y = 0.95, color = 'white', fontsize = 15)
        ax.set_title('Elasped Time: ' + str(float('%.6g' % time_span[i])) + ' yrs', color = 'white', fontsize  = 12)
        ax.set_xlim(min_x - mid_x * 0.05, max_x + mid_x * 0.05)
        ax.set_ylim(min_y - mid_y * 0.05, max_y + mid_y * 0.05)
        ax.set_zlim(min_z - mid_z * 0.05, max_z + mid_z * 0.05)
        ax.set_xlabel('X-axis (AU)', color = 'white', fontsize = 10)
        ax.set_ylabel('Y-axis (AU)', color = 'white', fontsize = 10)
        ax.set_zlabel('Z-axis (AU)', color = 'white', fontsize = 10)
        ax.tick_params(colors = 'white', which = 'both')
        
        ax.xaxis.pane.set_color((0, 0, 0, 1))
        ax.yaxis.pane.set_color((0, 0, 0, 1))
        ax.zaxis.pane.set_color((0, 0, 0, 1))
        
        plt.draw()
        plt.pause(0.01)
        writer.grab_frame()